In [20]:
import numpy as np
from numpy.typing import NDArray
from typing import Final

from minicombust import Cell, Face, Geometry

# SIMPLE with corrector steps


SIMPLE slves the momentum equations for velocity using a guess for pressure field and face mass fluxes, after which mass fluxes are updated with the new velocity values. But these will not satisfy pressure continuity equatins.
Flux correction a pressure correcton field.

[Mostsly from CompFDInINdustrialComnustion]

We use a sequential approach to solving the system of discretised PDEs (and particle tracking), i.e. we solve the governing equations one at a time. Picard iteration for non-linearities.

From Dolfyn source:
```
!     This routine assembles and solves the pressure-correction
!     equation using colocated grid. SIMPLE algorithm with one
!     or more corrector steps (non-orthogonality effects taken
!     into account as described in Sect. 8.8 of the corrected
!     2nd printing.
```

We use co-located pressure and velocity points (i.e. at same locations in grid)

What do we do to avoid chekerboarding? Add Rhie and Chow pressure dissipation term as in 8.8 of [Peric]

# Linear Solvers

Since MiniCombust relies on very large (very distributed) problems, the linear solvers used to solve the systems $A\mathbf{x} = \mathbf{b}$ (with A very sparse) must scale. As a results, we use iterative solvers.

PetSc

In addition, the discretise equations have non-linearities, so we must use several sweeps (updating the source terms and coefficients using current iteration values of $x$)

Unlike with structured grid/stencil problems, $A$ does not have a specific band struture.

Multigrid.

# Determining whether a particle is in a cell
We use the cell face normals (positive out) and a normalised position vector from the particle's position to the face centre.
Calculate dot product with cell face centre.
Result: 0 on face, positive inside, negative outside.

Check for all cell faces whether any one is negative.


# Calculating gradients


MiniCombust uses [Gauss's Divergence Theorem](https://en.wikipedia.org/wiki/Divergence_theorem) to calculate the gradients in diffusive flux terms.
The divergence theorem allows us to express the flux of a vector field through the surface in terms of the divergence of the field in the volume enclosed.


$$
\int_V (\nabla \cdot \mathbf{\Phi}) dV = \oint_S \mathbf{\Phi} d\vec{s}
$$

or in terms of discrete faces:

$$
(grad \mathbf{\Phi})_P \approx \frac{1}{V_P}\sum_{j=1}^{n}\Phi_j \vec{s_j}
$$

with $\Phi_j$ the value stored at the centre of face $j$

During gradient calculation, we also perform a deferred correction by adjusting the coordinates by weighting the adjacent cells' contributions using the face's interpolation factor property ($\lambda$).
This is described in [Peric](Resources.ipynb/#peric) in §8.6.2 (Approximation of Diffusive fluxes). 

Once the gradient has been calculated, we can apply an appropriate slope limiter, in case of bad meshes.

TODO: Discuss Slope limiters? Barth and Jespersen, Venkatakrishnan, Mavriplis, Aftosmis etc.

Refer to
https://core.ac.uk/download/pdf/10513971.pdf

Note that we limit the number of passes of gradient estimation to 2.

In [21]:
NumPassesOfGaussGradientEstimation: Final[int] = 2.

def gradient(mesh: Geometry, Φ: NDArray[np.float64]) -> NDArray[np.float64]:
    """

        Notes
        -----
        Based on GradientPhiGauss and GradientPhi in Dolfyn (gradients.f90)
    """
    assert(Φ.ndim == 1) # 1D array of length mesh.numCells + mesh.numBoundaries
    assert(len(Φ) ==  mesh.num_cells + mesh.num_boundaries)
    dΦdX = np.zeros((Φ.shape[0], 3)) # dΦdX will have 3 elems for every one of Φ
    dΦdX_corrections = np.zeros((Φ.shape[0], 3)) 

    
    for _ in range(NumPassesOfGaussGradientEstimation):
        for face in mesh.faces:
            cell1, cell2 = face.adjacent_cells
            is_boundary_face = cell2 is None
            if not is_boundary_face:
                λ_cell1, λ_cell2 = 1 - face.interpolation_factor, face.interpolation_factor
                # TODO is it worth precomputing this weighted corrected coord and storing in face?
                # Correction to face using interpolation factor 
                corrected_coords = cell1.coords * λ_cell1 + cell2.coords * λ_cell2 
                dΦdX_corrected = dΦdX_corrections[cell1.local_id, :] * λ_cell1 +  dΦdX_corrections[cell2.local_id, :] * λ_cell2 

                # Now gradient at shifted position is knonw
                # Correct the value at the cell face centre
                Φ_face = Φ[cell1.local_id] * λ_cell1 + Φ[cell2.local_id] * λ_cell2 # standard
                Δ = dΦdX_corrected.T * face.centre_coords - corrected_coords # corrected
                Φ_face += Δ

                # now only the value at the face center is known
                # multiply it by the area-components
                # this is basically Gauss' theorem
                dΦdX[cell1.local_id,:] = dΦdX[cell1.local_id,:] + Φ_face * face.normal_components
                dΦdX[cell2.local_id,:] = dΦdX[cell2.local_id,:] - Φ_face * face.normal_components

            else: # it's a boundary face
                Φ_face = Φ[mesh.num_cells + face.boundary_id]
                dΦdX[cell1.local_id, :] +=  Φ_face * face.normal_components
        dΦdX_corrections[:] = dΦdX
        # OR  for  under relaxation
        # dΦdX_corrections[:] = dΦdX_corrections[:] + 0.95*( dΦdX - dΦdX_corrections)

    # normalise by cell volume dΦdX 
    inv_cell_volumes = [1. / mesh.cells[i].volume for i in mesh.cells]
    dΦdX *= inv_cell_volumes

    return dΦdX


    